In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import csv
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import preprocessing
from hanspell import spell_checker
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric
from gensim.models.coherencemodel import CoherenceModel
import os

In [12]:
#한글 아닌 문자 삭제
#한글과 숫자를 제외한 모든 문자 삭제
def text_cleaning(text):
    #\xa0(유니코드) 삭제하기 
    text=text.strip("\xa0")
    hangul = re.compile('[^ 가-힣]+') #한글의 정규 표현식
    result = hangul.sub(' ', str(text))
    return result

def make_list(i):
    i = i.split(',')
    return i

In [13]:
files= pd.read_csv('reviews.csv' ,encoding='UTF8')
files = files.loc[1:130455]
files = files.drop(columns=['Unnamed: 0'], axis=1)
files = files.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
files = files.drop(columns=['index'], axis = 1)
files

,product_name,rating,skin_type,review
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,Dr.G 제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여...
1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,"👉🏻 간단정리! - good! : 자극없이 무난함, 봄여름가을에 적당한 수분감, 겨..."
2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,❤️나의 피부상태: 복합성+트러블 조금만 자극적이면 피부 뒤집어짐;💜장점: 여드름...
3,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,지성,또 샀어요!!!! 진짜 닥터지 최고입니다.. 건성분들이 쓰시기에는 너무 빨리 날아가...
4,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,"닥터지 크림은 워낙 유명하기도 하고,성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔..."
...,...,...,...,...
130450,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,최애 입니다이제 이것만 쓸거예요또사러 가야지 눈누난나 신난다
130451,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,자극이 없고 일단은 너무 촉촉해서 좋아요!이것만 쓰게 될것같아ㅛ
130452,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,배송도 빠르고 세일할 때 사야할거 같아서 샀어요송악크림 건성한테 정말 좋을거같아요
130453,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,가벼운제형이고 끈적거리지 않아서 좋았어요. 그리고 진정에도 좋습니다!


In [14]:
files = files.drop_duplicates(['review'])
files = files.reset_index(drop=True)
print(files)
files['review']=files['review'].apply(lambda x: text_cleaning(x))
#공백 제거
files['review'] = files['review'].str.strip()
idx = files[files['review']==""].index
files = files.drop(idx)

files = files.reset_index(drop=True)
#필요없이 생성된 column 제거
#files = files.drop(columns=['index'], axis=1)
#files = files.drop(columns=['Unnamed: 0'], axis=1)
print(files)

                                           product_name rating skin_type  \
0      [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
1      [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
2      [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
3      [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5        지성   
4      [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
...                                                 ...    ...       ...   
92586                        밀크터치 서양송악 진정크림+송악점보패드(60매)      5      None   
92587                        밀크터치 서양송악 진정크림+송악점보패드(60매)      5      None   
92588                        밀크터치 서양송악 진정크림+송악점보패드(60매)      5      None   
92589                        밀크터치 서양송악 진정크림+송악점보패드(60매)      5      None   
92590                        밀크터치 서양송악 진정크림+송악점보패드(60매)      5      None   

                                                  review  
0      Dr.G 제품은 정말 사용해보시라고 추

In [15]:
# Komoran을 이용해 한국어 기준 형태소로 토큰화
from konlpy.tag import Komoran
komoran = Komoran(userdic = 'user_dic.txt')
token = []
tokens = []

for i in files['review']:
    token_list = komoran.pos(i)
    #for i in token_list:
        #if i[1]=="NNP" or i[1]=="NNG" or i[1]=="NP":
            #token.append(i[0])
    tokens.append(token_list)

In [16]:
token = pd.DataFrame(columns=['review_tokens'])
token = noun_token.assign(review_tokens = tokens)
#noun_token = noun_token.transpose()
#vv_token.rename(columns = {0:'vv_token'}, inplace = True)
#noun_token.rename(columns = {0:'noun_token'}, inplace = True)
files = pd.concat([files, token], axis = 1)

files

,product_name,rating,skin_type,review,review_tokens,review_token
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여드름 고민...,"[(제품, NNG), (은, JX), (정말, MAG), (사용, NNG), (하,...","[(제품, NNG), (은, JX), (정말, MAG), (사용, NNG), (하,..."
1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,간단정리 자극없이 무난함 봄여름가을에 적당한 수분감 겨울엔 두겹 레이어링하면 쫀쫀 ...,"[(간단, XR), (정리, NNG), (자극, NNG), (없이, MAG), (무...","[(간단, XR), (정리, NNG), (자극, NNG), (없이, MAG), (무..."
2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,나의 피부상태 복합성 트러블 조금만 자극적이면 피부 뒤집어짐 장점 여드름 짜거나 엄...,"[(나, NP), (의, JKG), (피부, NNG), (상태, NNG), (복합성...","[(나, NP), (의, JKG), (피부, NNG), (상태, NNG), (복합성..."
3,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,지성,또 샀어요 진짜 닥터지 최고입니다 건성분들이 쓰시기에는 너무 빨리 날아가서 피부에 ...,"[(또, MAJ), (사, VV), (았, EP), (어요, EC), (진짜, MA...","[(또, MAJ), (사, VV), (았, EP), (어요, EC), (진짜, MA..."
4,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,닥터지 크림은 워낙 유명하기도 하고 성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔...,"[(닥터, NNP), (지, NNB), (크림, NNP), (은, JX), (워낙,...","[(닥터, NNP), (지, NNB), (크림, NNP), (은, JX), (워낙,..."
...,...,...,...,...,...,...
92460,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,최애 입니다이제 이것만 쓸거예요또사러 가야지 눈누난나 신난다,"[(최, XPN), (애, NNG), (입, NNG), (니다, NNP), (이제,...","[(최, XPN), (애, NNG), (입, NNG), (니다, NNP), (이제,..."
92461,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,자극이 없고 일단은 너무 촉촉해서 좋아요 이것만 쓰게 될것같아,"[(자극, NNG), (이, JKS), (없, VA), (고, EC), (일단, M...","[(자극, NNG), (이, JKS), (없, VA), (고, EC), (일단, M..."
92462,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,배송도 빠르고 세일할 때 사야할거 같아서 샀어요송악크림 건성한테 정말 좋을거같아요,"[(배, NNG), (송도, NNP), (빠르, VA), (고, EC), (세일, ...","[(배, NNG), (송도, NNP), (빠르, VA), (고, EC), (세일, ..."
92463,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,가벼운제형이고 끈적거리지 않아서 좋았어요 그리고 진정에도 좋습니다,"[(가볍, VA), (ㄴ, ETM), (제, XSN), (형, XSN), (이, V...","[(가볍, VA), (ㄴ, ETM), (제, XSN), (형, XSN), (이, V..."


In [17]:
files = files.loc[(files["skin_type"]=="None") | (files["skin_type"]=="복합성") | (files["skin_type"]=="지성") | (files["skin_type"]=="건성")]
files


,product_name,rating,skin_type,review,review_tokens,review_token
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여드름 고민...,"[(제품, NNG), (은, JX), (정말, MAG), (사용, NNG), (하,...","[(제품, NNG), (은, JX), (정말, MAG), (사용, NNG), (하,..."
1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,간단정리 자극없이 무난함 봄여름가을에 적당한 수분감 겨울엔 두겹 레이어링하면 쫀쫀 ...,"[(간단, XR), (정리, NNG), (자극, NNG), (없이, MAG), (무...","[(간단, XR), (정리, NNG), (자극, NNG), (없이, MAG), (무..."
2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,나의 피부상태 복합성 트러블 조금만 자극적이면 피부 뒤집어짐 장점 여드름 짜거나 엄...,"[(나, NP), (의, JKG), (피부, NNG), (상태, NNG), (복합성...","[(나, NP), (의, JKG), (피부, NNG), (상태, NNG), (복합성..."
3,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,지성,또 샀어요 진짜 닥터지 최고입니다 건성분들이 쓰시기에는 너무 빨리 날아가서 피부에 ...,"[(또, MAJ), (사, VV), (았, EP), (어요, EC), (진짜, MA...","[(또, MAJ), (사, VV), (았, EP), (어요, EC), (진짜, MA..."
4,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,닥터지 크림은 워낙 유명하기도 하고 성분이 순하고 진정효과가 좋아서 꾸준히 구매해왔...,"[(닥터, NNP), (지, NNB), (크림, NNP), (은, JX), (워낙,...","[(닥터, NNP), (지, NNB), (크림, NNP), (은, JX), (워낙,..."
...,...,...,...,...,...,...
92460,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,최애 입니다이제 이것만 쓸거예요또사러 가야지 눈누난나 신난다,"[(최, XPN), (애, NNG), (입, NNG), (니다, NNP), (이제,...","[(최, XPN), (애, NNG), (입, NNG), (니다, NNP), (이제,..."
92461,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,자극이 없고 일단은 너무 촉촉해서 좋아요 이것만 쓰게 될것같아,"[(자극, NNG), (이, JKS), (없, VA), (고, EC), (일단, M...","[(자극, NNG), (이, JKS), (없, VA), (고, EC), (일단, M..."
92462,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,배송도 빠르고 세일할 때 사야할거 같아서 샀어요송악크림 건성한테 정말 좋을거같아요,"[(배, NNG), (송도, NNP), (빠르, VA), (고, EC), (세일, ...","[(배, NNG), (송도, NNP), (빠르, VA), (고, EC), (세일, ..."
92463,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,가벼운제형이고 끈적거리지 않아서 좋았어요 그리고 진정에도 좋습니다,"[(가볍, VA), (ㄴ, ETM), (제, XSN), (형, XSN), (이, V...","[(가볍, VA), (ㄴ, ETM), (제, XSN), (형, XSN), (이, V..."


In [18]:
files['skin_type'].value_counts()

None    68008
복합성     12127
건성       7046
지성       3241
Name: skin_type, dtype: int64

In [19]:
from konlpy.tag import Komoran
komoran = Komoran(userdic = 'user_dic.txt')
a = "지성인데 복합성 같기도 하고 뭔가 아무튼 수부지일수도..."
token_list = komoran.pos(a)

In [20]:
token_list

[('지성', 'NNG'),
 ('이', 'VCP'),
 ('ㄴ데', 'EC'),
 ('복합성', 'NNG'),
 ('같', 'VA'),
 ('기', 'ETN'),
 ('도', 'JX'),
 ('하', 'VV'),
 ('고', 'EC'),
 ('뭐', 'NP'),
 ('ㄴ가', 'EC'),
 ('아무튼', 'MAG'),
 ('수부지', 'NNG'),
 ('일수', 'NNP'),
 ('도', 'JX'),
 ('...', 'SE')]

In [21]:
def case1_a(review):

  # 저는~
    for i in range(len(review)-3):

        if review[i][0]=='저' and review[i+1][0]=='는' and\
            review[i+2][0]=='건성' and review[i+3][1]=='VCP':
            return '건성'

        elif review[i][0]=='저' and review[i+1][0]=='는' and\
            review[i+2][0]=='지성' and review[i+3][1]=='VCP':
            return '지성'

        elif review[i][0]=='저' and review[i+1][0]=='는' and\
            review[i+2][0]=='복합성' and review[i+3][1]=='VCP':
            return '복합성'

        elif review[i][0]=='저' and review[i+1][0]=='는' and\
            review[i+2][0]=='수부지' and review[i+3][1]=='VCP':
            return '지성'

  # 전~
    for i in range(len(review)-2):

        if review[i][0]=='전' and review[i+1][0]=='건성' and review[i+2][1]=='VCP':
            return '건성'

        elif review[i][0]=='전' and review[i+1][0]=='지성' and review[i+2][1]=='VCP':
            return '지성'

        elif review[i][0]=='전' and review[i+1][0]=='복합성' and review[i+2][1]=='VCP':
            return '복합성'

        elif review[i][0]=='전' and review[i+1][0]=='수부지' and review[i+2][1]=='VCP':
            return '지성'

    return 'None'

In [22]:
def case1_b(tokens):
    for j in range(len(tokens)-2):
        if tokens[j][0] =='제가' and tokens[j+1][0]=='건성' and tokens[j+2][1] == 'VCP':
            return '건성'
        elif tokens[j][0] =='제가' and tokens[j+1][0]=='지성' and tokens[j+2][1] == 'VCP':
            return '지성'
        elif tokens[j][0] =='제가' and tokens[j+1][0]=='복합성' and tokens[j+2][1] == 'VCP':
            return '복합성'
        elif tokens[j][0] =='제가' and tokens[j+1][0]=='수부지' and tokens[j+2][1] == 'VCP':
            return '지성'

    return 'None'

In [23]:
def case2_a(review):
    for i in tqdm(range(len(review))):
        if i<len(review)-8:
            if review[i][0]=='저' and (review[i+1][0]=="의" or review[i+1][0] =='는') and (review[i+2][0] == '피부타입' or review[i+2][0] == '피부') and (review[i+3][1] == 'JX' or review[i+3][1] == 'JKS')and review[i+4][0] == '복합성' and ((review[i+5][1] == 'JKB' or review[i+5][1] == 'VCP') or (review[i+5][1] == 'NNG' and (review[i+6][1]=='JKB' or review[i+6][1]=='VCP'))):
                type_skin = "복합성"
                return type_skin
            
            elif review[i][0]=='저' and (review[i+1][0]=="의" or review[i+1][0] =='는') and (review[i+2][0] == '피부타입' or review[i+2][0] == '피부') and (review[i+3][1] == 'JX' or review[i+3][1] == 'JKS')and review[i+4][0] == '지성' and ((review[i+5][1] == 'JKB' or review[i+5][1] == 'VCP') or (review[i+5][1] == 'NNG' and (review[i+6][1]=='JKB' or review[i+6][1]=='VCP'))):
                type_skin = "지성"
                return type_skin
            
            elif review[i][0]=='저' and (review[i+1][0]=="의" or review[i+1][0] =='는') and (review[i+2][0] == '피부타입' or review[i+2][0] == '피부') and (review[i+3][1] == 'JX' or review[i+3][1] == 'JKS')and review[i+4][0] == '건성' and ((review[i+5][1] == 'JKB' or review[i+5][1] == 'VCP') or (review[i+5][1] == 'NNG' and (review[i+6][1]=='JKB' or review[i+6][1]=='VCP'))):
                type_skin = "건성"
                return type_skin
            
            elif review[i][0]=='저' and (review[i+1][0]=="의" or review[i+1][0] =='는') and (review[i+2][0] == '피부타입' or review[i+2][0] == '피부') and (review[i+3][1] == 'JX' or review[i+3][1] == 'JKS')and review[i+4][0] == '수부지' and ((review[i+5][1] == 'JKB' or review[i+5][1] == 'VCP') or (review[i+5][1] == 'NNG' and (review[i+6][1]=='JKB' or review[i+6][1]=='VCP'))):
                type_skin = "지성"
                return type_skin
            
            elif review[i][0]=='제' and (review[i+1][0]=="피부타입" or review[i+1][0] =='피부') and (review[i+2][1] == 'JX' or review[i+2][1] == 'JKS') and review[i+3][0] == '복합성' and ((review[i+4][1] == 'JKB' or review[i+4][1] == 'VCP') or (review[i+4][1] == 'NNG' and (review[i+5][1]=='JKB' or review[i+5][1]=='VCP'))):
                type_skin = "복합성"
                return type_skin
            
            elif review[i][0]=='제' and (review[i+1][0]=="피부타입" or review[i+1][0] =='피부') and (review[i+2][1] == 'JX' or review[i+2][1] == 'JKS')and review[i+3][0] == '건성' and ((review[i+4][1] == 'JKB' or review[i+4][1] == 'VCP') or (review[i+4][1] == 'NNG' and (review[i+5][1]=='JKB' or review[i+5][1]=='VCP'))):
                type_skin = "건성"
                return type_skin
            
            elif review[i][0]=='제' and (review[i+1][0]=="피부타입" or review[i+1][0] =='피부') and (review[i+2][1] == 'JX' or review[i+2][1] == 'JKS')and review[i+3][0] == '지성' and ((review[i+4][1] == 'JKB' or review[i+4][1] == 'VCP') or (review[i+4][1] == 'NNG' and (review[i+5][1]=='JKB' or review[i+5][1]=='VCP'))):
                type_skin = "지성"
                return type_skin
            
            elif review[i][0]=='제' and (review[i+1][0]=="피부타입" or review[i+1][0] =='피부') and (review[i+2][1] == 'JX' or review[i+2][1] == 'JKS')and review[i+3][0] == '수부지' and ((review[i+4][1] == 'JKB' or review[i+4][1] == 'VCP') or (review[i+4][1] == 'NNG' and (review[i+5][1]=='JKB' or review[i+5][1]=='VCP'))):
                type_skin = "지성"
                return type_skin
                       
        else:
            type_skin = "None"
            return type_skin

In [24]:
def case3_a(review):
    for i in range(len(review)):
        if i<len(review)-1:
            if review[i][0]=='피부타입' and review[i+1][0]=="복합성" :
                type_skin = "복합성"
                return type_skin
            elif review[i][0]=="피부타입" and review[i+1][0]=="지성":
                type_skin = "지성"
                return type_skin
            elif review[i][0]=="피부타입" and review[i+1][0]=="수부지":
                type_skin = "지성"
                return type_skin
            elif review[i][0]=="피부타입" and review[i+1][0]=="건성":
                type_skin = "건성"
                return type_skin
        else:
            type_skin = "None"
            return type_skin

In [25]:
def case3_b(review):
    for i in range(len(review)):
        if i<len(review)-1:
            if review[i][0]=='피부' and review[i+1][0]=="복합성" :
                type_skin = "복합성"
                return type_skin
            elif review[i][0]=="피부" and review[i+1][0]=="지성":
                type_skin = "지성"
                return type_skin
            elif review[i][0]=="피부" and review[i+1][0]=="수부지":
                type_skin = "지성"
                return type_skin
            elif review[i][0]=="피부" and review[i+1][0]=="건성":
                type_skin = "건성"
                return type_skin
        else:
            type_skin = "None"
            return type_skin

In [26]:
#Test 

from konlpy.tag import Komoran
komoran = Komoran(userdic = 'user_dic.txt')
a = "지성인데 복합성 같기도 하고 뭔가 아무튼 수부지일수도..."
token_list1 = komoran.pos(a)
review = "쓰니피부: 건성 반갑습니다. 저는 이수진입니다."
review = text_cleaning(review)
token_list2 = komoran.pos(review)
print(case3_b(token_list1))
print(case3_b(token_list2))

None
건성


In [27]:
from konlpy.tag import Komoran

from tqdm import tqdm

files['skin_type'].loc[files['skin_type']=="None"] = files['review_tokens'].loc[files['skin_type']=="None"].apply(lambda x: case3_b(x))
files['skin_type'].loc[files['skin_type']=="None"] = files['review_tokens'].loc[files['skin_type']=="None"].apply(lambda x: case3_a(x))
files['skin_type'].loc[files['skin_type']=="None"] = files['review_tokens'].loc[files['skin_type']=="None"].apply(lambda x: case1_a(x))
files['skin_type'].loc[files['skin_type']=="None"] = files['review_tokens'].loc[files['skin_type']=="None"].apply(lambda x: case1_b(x))
files['skin_type'].loc[files['skin_type']=="None"] = files['review_tokens'].loc[files['skin_type']=="None"].apply(lambda x: case2_a(x))


/Users/soojin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 62%|██████▏   | 13/21 [00:00<00:00, 315178.91it/s]


 95%|█████████▌| 164/172 [00:00<00:00, 330545.82it/s]


 95%|█████████▌| 167/175 [00:00<00:00, 830900.08it/s]


 94%|█████████▍| 135/143 [00:00<00:00, 746022.45it/s]


 71%|███████▏  | 20/28 [00:00<00:00, 376170.76it/s]


...

 91%|█████████ | 81/89 [00:00<00:00, 599186.29it/s]


 33%|███▎      | 4/12 [00:00<00:00, 125203.10it/s]


 91%|█████████▏| 84/92 [00:00<00:00, 800730.76it/s]


 95%|█████████▍| 151/159 [00:00<00:00, 811974.24it/s]


 58%|█████▊    | 11/19 [00:00<00:00, 299593.14it/s]


In [28]:
files['skin_type'].value_counts()

None    65981
복합성     12655
건성       7845
지성       3941
Name: skin_type, dtype: int64

In [29]:
files.to_csv("type_classification_tokens.csv", header = False)

In [ ]:
from konlpy.tag import Komoran
komoran = Komoran()
reviews = []
nouns = []
review_all = []
review_noun = []
for i in noun_token['review_token']:
    for a in i:
        if a[1]=="NNG" or a[1]=="NNP":
            nouns.append(a[0])
    review_noun.append(nouns)
    nouns = []

noun_token['noun_token'] = review_noun

In [ ]:
#2, 3-gram
list_2_gram = []
list_3_gram = []
noun_2_gram = []
noun_3_gram = []
alls = []

#명사로만 token (연속단어 선택 시 사용)
for i in noun_token['noun_token']:
    for a in zip(i, i[1:]):
        noun_2_gram.append(str(a[0]+a[1]))
    for b in zip( *[i[j:] for j in range(3)] ):
        noun_3_gram.append(str(b[0]+b[1]+b[2]))

df_noun_2 = pd.DataFrame({'2gram':noun_2_gram})
df_noun_3 = pd.DataFrame({'3gram':noun_3_gram})

In [ ]:
bigram = df_noun_2['2gram'].value_counts()
bigram = bigram.sort_values(ascending = False)
count=0
for idx, value in zip(bigram.index, bigram.values):
    print(idx, value)
    count+=1
    if count>101:
        break

In [ ]:
trigram = df_noun_3['3gram'].value_counts()
trigram = trigram.sort_values(ascending = False)
count = 0
for idx, value in zip(trigram.index, trigram.values):
    print(idx, value)
    count+=1
    if count>101:
        break